In [62]:
import pandas as pd
import duckdb as db

carpeta = "~/Documents/university/laboratorio_de_datos/TP/"

datos_por_departamento = pd.read_csv(carpeta+"Datos_por_departamento_actividad_y_sexo.csv")
padron_poblacion = pd.read_csv(carpeta+"padron_poblacion.xlsX - Output.csv")
actvidades_establecimiento = pd.read_csv(carpeta+"actividades_establecimientos.csv")
padron_oficial_establecimientos_educativos_2022 = pd.read_csv(carpeta+"2022_padron_oficial_establecimientos_educativos.csv", skiprows=6)


In [64]:
modalidad_educativa = pd.DataFrame(columns=["id_modalidad", "modalidad", "sub_modalidad"])
establecimiento_educativo = pd.DataFrame(columns=["CUE_establecimiento", "nombre", "sector", "mail", "telefono", "codigo_area"])
establecimiento_modalidad = pd.DataFrame(columns=["CUE_establecimiento", "id_modalidad"])
establecimiento_ubicacion = pd.DataFrame(columns=["CUE_establecimiento", "id_departamento", "ambito", "domicilio", "codigo_postal", "localidad"])
departamento_catalogo = pd.DataFrame(columns=["id_departamento", "id_provincia", "departamento"])
provincia_catalogo = pd.DataFrame(columns=["id_provincia", "provincia"])
actividad_departamento = pd.DataFrame(columns=["id_departamento", "clae6", "genero", "empleo", "establecimiento", "empresas_exportadoras"])
actividad_catalogo = pd.DataFrame(columns=["clae6", "descripcion"])
poblacion_departamento = pd.DataFrame(columns=["id_departamento", "edad", "casos"])

query1 = """
        SELECT DISTINCT in_departamentos AS id_departamento, provincia_id AS id_provincia, departamento AS departamento
        FROM datos_por_departamento
"""

departamento_catalogo = db.query(query1)

query2 = """
        SELECT DISTINCT provincia_id AS id_provincia, provincia AS provincia 
        FROM datos_por_departamento
"""

provincia_catalogo = db.query(query2)

query3 = """
        SELECT DISTINCT clae6, clae6_desc FROM actvidades_establecimiento
"""

actividad_catalogo = db.query(query3)

query4 = """
        SELECT "Jurisdicción", "Cueanexo", "Nombre", "Sector", "Ámbito", "Domicilio", "C. P.", "Código de área", "Teléfono", "Código de localidad", "Localidad", "Departamento", "Mail", 
        "Nivel inicial - Jardín maternal" AS "Comun-Nivel inicial - Jardín maternal", "Nivel inicial - Jardín de infantes" AS "Comun-Nivel inicial - Jardín de infantes",
        "Primario" AS "Comun-Primario", "Secundario" AS "Comun-Secundario", "Secundario - INET" AS "Comun-Secundario - INET", "SNU" AS "Comun-SNU", "SNU - INET" AS "Comun-SNU - INET",
        "Intercultural", "Encierro",
        "Secundario.1" AS "Artística-Secundario", "SNU.1" AS "Artística-SNU", "Talleres" AS "Artística-Talleres", 
        "Nivel inicial - Educación temprana" AS "Especial-Nivel inicial - Educación temprana", "Nivel inicial - Jardín de infantes" AS "Especial-Nivel inicial - Jardín de infantes",
        "Primario.1" AS "Especial-Primario", "Secundario.2" AS "Especial-Secundario", "Integración a la modalidad común/ adultos" AS "Especial-Integración a la modalidad común/ adultos",
        "Integración a la modalidad común/ adultos" AS "Adultos-Integración a la modalidad común/ adultos", "Primario.2" AS "Adultos-Primario", "Secundario.3" AS "Adultos-Secundario",
        "Alfabetización" AS "Adultos-Alfabetización", "Formación Profesional" AS "Adultos-Formación Profesional", "Formación Profesional - INET" AS "Adultos-Formación Profesional - INET",
        "Inicial" AS "Hospitalaria-Inicial", "Primario.3" AS "Hospitalaria-Primario", "Secundario.4" AS "Hospitalaria-Secundario" FROM padron_oficial_establecimientos_educativos_2022
"""

padron_oficial_establecimientos_educativos_2022 = db.query(query4)
padron_oficial_establecimientos_educativos_2022

┌────────────────────────┬──────────┬────────────────────────────────────────────────────────────────────┬─────────┬─────────┬────────────────────────────────────────────────┬─────────┬────────────────┬─────────────────────┬─────────────────────┬────────────────────────┬──────────────┬───────────────────────────────────────────────────────────────────────────────┬───────────────────────────────────────┬──────────────────────────────────────────┬────────────────┬──────────────────┬─────────────────────────┬───────────┬──────────────────┬───────────────┬──────────┬──────────────────────┬───────────────┬────────────────────┬─────────────────────────────────────────────┬─────────────────────────────────────────────┬───────────────────┬─────────────────────┬────────────────────────────────────────────────────┬───────────────────────────────────────────────────┬──────────────────┬────────────────────┬────────────────────────┬───────────────────────────────┬───────────────────────────────────